## Получение данных

### Загрузка и просмотр данных

прочитаем файл `data.csv` и результат сохраним в переменной `borrowers`. Получим 10 случайных строк таблицы


In [ ]:
import pandas as pd

borrowers = pd.read_csv('/content/data.csv')
borrowers.sample(10)



,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
15692,2,-81.571681,56,среднее,1,Не женат / не замужем,4,F,сотрудник,0,163533.194058,строительство собственной недвижимости
13636,0,-3794.440187,56,ВЫСШЕЕ,0,женат / замужем,0,F,сотрудник,0,80665.513551,строительство собственной недвижимости
9814,3,NaN,31,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,свой автомобиль
16224,0,-2793.328819,48,среднее,1,женат / замужем,0,F,сотрудник,0,117638.391094,строительство собственной недвижимости
17632,0,-11485.419727,63,высшее,0,женат / замужем,0,M,компаньон,0,193660.935812,операции с недвижимостью
19573,0,-929.122673,43,среднее,1,гражданский брак,1,F,компаньон,0,333948.163599,покупка жилья для сдачи
9213,1,-1646.841004,21,среднее,1,женат / замужем,0,F,госслужащий,0,81570.579349,приобретение автомобиля
8827,1,-516.050288,45,среднее,1,женат / замужем,0,F,компаньон,0,103257.421241,автомобили
2683,0,-3442.298864,38,среднее,1,Не женат / не замужем,4,M,компаньон,0,255223.578031,получение высшего образования
13310,1,-3826.557851,27,Среднее,1,женат / замужем,0,F,сотрудник,0,141596.376352,покупка недвижимости


Узнаем уровень разброса значений в исследуемых данных `borrowers`. Для этого применим метод describe()

In [ ]:
borrowers.describe()

Получим общую информацию о данных таблицы `borrowers`, применив метод info()

In [ ]:
borrowers.info()

**Рассмотрим полученную информацию подробнее**

Всего в таблице 12 столбцов, тип данных столбцов различается. Разбереме подробно, какие в `borrowers` столбцы и какую информацию они содержат:

- *children* — количество детей в семье  
- *days_employed* — общий трудовой стаж в днях  
- *dob_years* — возраст клиента в годах  
- *education* — уровень образования клиента  
- *education_id* — идентификатор уровня образования  
- *family_status* — семейное положение  
- *family_status_id* — идентификатор семейного положения  
- *gender* — пол клиента  
- *income_type* — тип занятости  
- *debt* — имел ли задолженность по возврату кредитов  
- *total_income* — ежемесячный доход  
- *purpose* — цель получения кредита  




### **Выводы**

  
Каждая строка таблицы содержит информацию об определенном физическом лице, запрашивающем кредитные средства на известную цель. Из таблицы можем видеть семейное положение, количество дейтей, возрат, трудовой стаж. Исходя из количества данных, можно сделать вывод что имеем отсутствующие значения в столбцах `days_employed` и `total_income`. Благодаря методу `describe()` мы выявили, что в столбце `children` находится min значение - 1 и значение max для этого столбца 20, что отклоняется от общего. В столбце `days_employed`мы наблюдаем отрицательные значения. Также необходимо проверить данный столбце на наличие артефактов, так как среднее по столбцу - `63046.497661` дня. Также минимальное значение возраста значится как "0".

Необходимо:

- проверить столбец `children` на наличие артефактов и некорректных значений
- проверить наличие минусовых значений в столбце `days_employed` и выявить характер данного явления
- убрать пропуски данных
- проверить наличие дупликатов
- привести к единому регистру столбец `education` и `family_status`
- столбцам `days_employed` и `total_income` присвоить целоцисленные значения Int, это не повлияет на анализ, но упростит обработку
- исправить значение возраста 0

Попробуем решить данные задачи и тогда сможем приступить к обработке данных


## Предобработка данных

### Подсчет пропусков


Подсчитаем количество пропусков в сете `borrowers`

In [ ]:
borrowers.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Пропуски обнаружены в столбцах `days_employed` и `total_income`. Они носят неслучайный характер. если люди не работают, то и дохода, по крайней мере, официального не получают. Проверим, в одних и тех же строках наблюдаются значения NaN?

In [ ]:
# вызовем метод info () при одновременном выполнении двух условий:
borrowers[(borrowers['total_income'].isnull() == True) & (borrowers['days_employed'].isnull() == True)].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2174 entries, 12 to 21510
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          2174 non-null   int64  
 1   days_employed     0 non-null      float64
 2   dob_years         2174 non-null   int64  
 3   education         2174 non-null   object 
 4   education_id      2174 non-null   int64  
 5   family_status     2174 non-null   object 
 6   family_status_id  2174 non-null   int64  
 7   gender            2174 non-null   object 
 8   income_type       2174 non-null   object 
 9   debt              2174 non-null   int64  
 10  total_income      0 non-null      float64
 11  purpose           2174 non-null   object 
dtypes: float64(2), int64(5), object(5)
memory usage: 220.8+ KB


Соответственно, пропуски наблюдаются в одних и тех же строках. Это могут быть неработающие студенты, люди в отпуске по уходу за ребенком. Перед тем как перейти к удалению пропусков, необходимо разобраться со столбцами `children` и `days_employed`

### Изучение столбцов `children` и `days_employed`
#### столбец `children`

Для начала давайте разберемся "артефактами" столбца `children`
Для наглядности выведем первые 10 строк нашей таблицы

In [ ]:
borrowers.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Посчитаем количество людей с необычным количеством детей:

In [ ]:
#подсчет количества с -1 ребенком
print('Количество людей с "- 1" ребенком:', borrowers[borrowers['children'] == -1].count()[0])

#подсчет количества с 20 детьми
print('Количество людей с "20" детьми: ', borrowers[borrowers['children'] == 20].count()[0])

Количество людей с "- 1" ребенком: 47
Количество людей с "20" детьми:  76


Скорее всего, данные попали ошибочно в таблицу, либо это означает утрату ребенка. Для анализа "-1" исправим на "0", "20" не исправляем, чтобы не искажать статистику - будем считать, что это "многодетные" <a id="#category"> </a>:

In [ ]:
borrowers['children'] = borrowers['children'].replace(-1,0) # замена -1 ребенка на 0
print('Количество людей с "- 1" ребенком: ', borrowers[borrowers['children'] == -1].count()[0]) # Проверка замены



Количество людей с "- 1" ребенком:  0


#### столбец `days_employed`

Проверим количество строк со значениями больше или равное 0, и строк со значением меньше 0. Также посмотрим, сколько пенсионеров имеют отрицательный стаж работы.

In [ ]:
print('Количество строк трудового стажа больше или равное 0:', borrowers[borrowers['days_employed'] >= 0].shape[0])
print('Количество строк трудового стажа меньше 0:', borrowers[borrowers['days_employed'] < 0].shape[0])

#поместим в переменнную двойное условие: пенсионеры и трудовой стаж больше или равное 0

borrowers_oldest = borrowers[(borrowers['days_employed'] >= 0)&(borrowers['income_type'] == 'пенсионер')].shape[0]
print('Количество пенсионеров с трудовым стажем больше или равное 0:', borrowers_oldest)




Количество строк трудового стажа больше или равное 0: 3445
Количество строк трудового стажа меньше 0: 15906
Количество пенсионеров с трудовым стажем больше или равное 0: 3443


Получаем, что почти все положительные значения относятся к пенсионерам, соответственно полагаем, что весь трудовой стаж можно принимать как "положительный", минусовые значения появились ошибочно либо по неизвестному алгоритму, на анализ это столбец влияние не окажет. Произведем замену

In [ ]:
# применим метод abs() для получения значений по модулю
borrowers ['days_employed'] = borrowers ['days_employed'].abs()

# вызовем метод describe() для проверки 
borrowers.describe() 

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.541092,66914.728907,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.379943,139030.880527,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,0.000000,24.141633,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,927.009265,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,2194.220567,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,5537.882441,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


**Отрицательных значений в таблице нет**

### Обработка пропусков и нулевых значений


В результате применения метода `describe()` у столбца `days_employed` наблюдается явное отклонение. Посчитаем среднее в годах:

In [ ]:
# объявим константу
DAYS_IN_YEAR = 365

# выведем средний трудовой стаж в годах

print('Средний трудовой стаж в годах: {:.2f}'.format((borrowers['days_employed'].mean())/DAYS_IN_YEAR))



Средний трудовой стаж в годах: 183.33


Выявлены отклонения, из-за наличия которых, заменять NaN средним не стоит при таких данных в рамках всего сета. Давайте найдем другой путь. Например, попробуем заменить NaN на медианные значения. Проверим.

Согласно ст. [63](http://tkodeksrf.ru/ch-3/rzd-3/gl-11/st-63-tk-rf)
ТК РФ "В общем случае заключение трудового договора допускается с работниками в возрасте не моложе 16 лет". Попробуем рассчитать максимально возможный трудовой стаж для каждого возраста. 

Для этого добавим в таблицу дополнительный столбец `days_employed_ratio`, который будет отражать отношение трудового стажа из данных к рассчетному теоритечески возможному трудовому стажу `(borrowers['dob_years'] - 16) *  DAYS_IN_YEAR`

*Примечение частный случай 15 лет рассматривать не будет*

In [ ]:
borrowers['days_employed_ratio'] = borrowers ['days_employed'] / ((borrowers['dob_years'] - 16) *  DAYS_IN_YEAR)
borrowers.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,days_employed_ratio
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,0.889112
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,0.551343
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,0.906273
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,0.706292
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,25.195563
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья,0.230681
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем,0.292156
7,0,152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование,0.012311
8,2,6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы,0.999260
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи,0.239864


Если оценивать рейтинг, складывается впечатление, что верным решением будет заменить значение в столбце `days_employed` на теоретическое значение трудового стажа, если рейтинг выше 1, но учтем данный факт в последcтвии при категорировании, чтобы не искажать данные. Заменять данные не будем, ограничимся заменой NaN на медиану, так как у нас есть артефакты в данных и замена на среднее сильно исказит результат, вспомогательный столбец удалим

In [ ]:
# переведем стаж в года
borrowers['days_employed'] = borrowers['days_employed'].apply(lambda x: x/DAYS_IN_YEAR)

# посчитаем медиану для стажа и для ежемесячного дохода

borrowers_median = borrowers['days_employed'].median() 
stat_total_median = borrowers['total_income'].median()

#заполним пропуски медианой

borrowers['days_employed'] = borrowers['days_employed'].fillna(value=borrowers_median)
borrowers['total_income'] = borrowers['total_income'].fillna(value=stat_total_median)

#удалим столбец days_employed_ratio за ненадобностью:

borrowers = borrowers.drop('days_employed_ratio', axis=1)

#переименуем столбец days_employed на years_employed

borrowers = borrowers.rename(columns={'days_employed':'years_employed'})

#получим информацию о таблице после изменений:

borrowers.info()
borrowers.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   years_employed    21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,years_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,23.116912,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,11.026860,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,15.406637,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,11.300677,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,932.235814,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


#### **Вывод**

NaN-значения решили заменить на медиану, так как разброс данных велик, замену столбцов решили не приводить, рейтинг стажа удалили, столбец дней переименовали в года. Согласно последнему вызову метода info - NaN-значения отсутствуют. Переходим к замене типа данных

### Замена типа данных

Замени тип данных в столбцах `years_employed` и `total_income` на **int64**. Сейчас столбцы представлены в виде **float64**. Для этого вызовем метод `astype()`

In [ ]:
borrowers['years_employed'] = borrowers ['years_employed'].astype('int64')
borrowers['total_income'] = borrowers ['total_income'].astype('int64')
borrowers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int64 
 1   years_employed    21525 non-null  int64 
 2   dob_years         21525 non-null  int64 
 3   education         21525 non-null  object
 4   education_id      21525 non-null  int64 
 5   family_status     21525 non-null  object
 6   family_status_id  21525 non-null  int64 
 7   gender            21525 non-null  object
 8   income_type       21525 non-null  object
 9   debt              21525 non-null  int64 
 10  total_income      21525 non-null  int64 
 11  purpose           21525 non-null  object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


Осталось изменить регистр столбца `education` и `family_status`, приведем все буквы к нижнему регистру

In [ ]:
borrowers['education'] = borrowers['education'].str.lower()
borrowers['family_status'] = borrowers['family_status'].str.lower()

borrowers.head(10)

,children,years_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,23,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,11,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,15,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,11,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,932,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,2,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0,7,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
7,0,0,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,18,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,5,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи


#### **Вывод**

Замену произвели все данные представлены в виде int64, что позволит видеть целочисленные значения, либо object. NaN убрали, регистр столбца `education` привели к единому, можем приступать к обработке дубликатов

### Обработка дубликатов

Установим наличие дубликатов в сете данных. Если найдутся - удаляем и проверим все ли удалили.

In [ ]:
borrowers.duplicated().sum()


71

Выявили по всему сету 71 дубликат. Дубликаты могли появится в результате сбоя записи данных. Стоит обратить на это внимание и разобраться с причинами появления. Удалим дубликаты, сброим индекс

In [ ]:
borrowers = borrowers.drop_duplicates().reset_index(drop=True) # удаляем дубликаты
borrowers.duplicated().sum() #проверяем их наличие снова

0

#### **Вывод**

Произвели замену дубликатов. Может приступать к дальнейшей обработке. Нам необходимо выделить в столбце `purpose` уникальные слова

### Лемматизация

В результате рассмотрения сета данных, в столбце `purpose` наблюдались схожие цели, но записанные разными выражениями. Для дальней категоризации нам необходимо привести их к единообразию. Для этого впосользуемся модулем `Pymystem3`. 

Ознакомимся со столбцом `purpose` более подробно.

In [ ]:
 # посчитаем уникальные значения столбца Purpose
unique_purposes = borrowers['purpose'].value_counts()

# выведем на экран количество тех или иных запросов


print(borrowers['purpose'].value_counts().sort_values(ascending=False))



свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
покупка жилья                             646
жилье                                     646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

In [ ]:
# получим лемматизатор\стеммер из модуля Pymystem3

from pymystem3 import Mystem

# вызовем коллекцию Counter

from collections import Counter


m = Mystem()
list_of_lemmas = []
for purpose in borrowers['purpose']:
    lemma = m.lemmatize(purpose)
    list_of_lemmas.extend(lemma)
    
# подсчитаем уникалльные леммы

unique_lemmas = Counter(list_of_lemmas)

# отсортируем уникальные значения

sorted(unique_lemmas.items(), key = lambda pair: pair[1], reverse=True)








[(' ', 33570),
 ('\n', 21454),
 ('недвижимость', 6351),
 ('покупка', 5897),
 ('жилье', 4460),
 ('автомобиль', 4306),
 ('образование', 4013),
 ('с', 2918),
 ('операция', 2604),
 ('свадьба', 2324),
 ('свой', 2230),
 ('на', 2222),
 ('строительство', 1878),
 ('высокий', 1374),
 ('получение', 1314),
 ('коммерческий', 1311),
 ('для', 1289),
 ('жилой', 1230),
 ('сделка', 941),
 ('дополнительный', 906),
 ('заниматься', 904),
 ('проведение', 768),
 ('сыграть', 765),
 ('сдача', 651),
 ('семья', 638),
 ('собственный', 635),
 ('со', 627),
 ('ремонт', 607),
 ('подержанный', 486),
 ('подержать', 478),
 ('приобретение', 461),
 ('профильный', 436)]

Получили список лемм, встречающихся в столбце `purpose`. Для выделения четких категорий, давайте оставим в списке только существительные, исключим пробелы, предлоги и прилагательные.

In [ ]:
#исключим из списка unique_lemmas слова, короче 2 символов (для исключения предлогов и пробелов) и получим новый список
list_of_purposes = [length for length in sorted(unique_lemmas, key=unique_lemmas.get, reverse=True)
                          if len(length) > 2]
print(list_of_purposes)

['недвижимость', 'покупка', 'жилье', 'автомобиль', 'образование', 'операция', 'свадьба', 'свой', 'строительство', 'высокий', 'получение', 'коммерческий', 'для', 'жилой', 'сделка', 'дополнительный', 'заниматься', 'проведение', 'сыграть', 'сдача', 'семья', 'собственный', 'ремонт', 'подержанный', 'подержать', 'приобретение', 'профильный']


Исключим из списка прилагательные и схожие по значению цели, остается:

**недвижимость, жилье, автомобиль, образование, операция, свадьба, строительство, ремонт**


*Примечание **жилье** пришлось включать, так как часто встречается и исключение "жил" не позволит точно категорировать*




Создадим новый список и из него соберем словарь

In [ ]:
list_of_purpose = {'недвиж':1, 'авто':2, 'образ':3, 'опер':4, 'свад':5, 'строит':6, 'ремо':7, 'жил':1}
# применим словрь к нашему сету и создадим столбец с категориями товара

borrowers['purpose_category'] = 0

#напишем цикл, который будет принимать значение из словаря и подставлять  столбец 11 сета, 
#и если слова из словаря находится в столбце 11 - выставлять категорию в столбец 12. Применим iloc

for row in range(len(borrowers)):
    for purpose in list_of_purpose:
        if purpose in borrowers.iloc[row, 11]:
            borrowers.iloc[row, 12] = list_of_purpose[purpose]
        
borrowers.head(10)

,children,years_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category
0,1,23,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,1
1,1,11,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,2
2,0,15,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,1
3,3,11,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,3
4,0,932,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,5
5,0,2,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,1
6,0,7,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,1
7,0,0,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,3
8,2,18,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,5
9,0,5,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,1


In [ ]:
#проверим количество 0 в новом столбце, если все верно - они должны отсутствовать
borrowers[borrowers['purpose_category'] == 0].count().sum()


0

#### **Вывод**

По результатм лемматизации столбца `purpose` выявили топ целей для получения кредитов

  - недвижимость
  - автомобиль
  - образование
  - операция
  - свадьба
  - строительство/ремонт
  
  
  Вполне ожидаемые цели, связанные с бытовой необходимостью. Можем переходить к категорированию данных и ответу на поставленные вопросы

### Категоризация данных

Перед нами поставлены следующая задача:

влияет ли семейное положение и количество детей клиента на факт погашения кредита.


Разобьем категории (#category) следующим образом: 

- по возрасту (`dob_years`):  

    - до 35 лет ([молодежь](https://www.vedomosti.ru/society/news/2020/11/11/846620-gosduma-odobrila-v-pervom-chtenii-povishenie-vozrasta-molodezhi-do-35-let))  
    - 36-45 лет  
    - 46-65 лет  
    - более 65 лет  
    
    

- по стажу (`year_employed`):  

    - до 10 лет  
    - 10-30 лет  
    - более 30 лет  
    
    
- по количеству детей (`children`):  

    - нет детей  
    - 1-2 ребенка  
    - многодетные (больше 3 детей)
    
    
- по [доходу](https://www.factograph.info/a/30497993.html) (`total_income`):  

    - низкий доход (менее 50000)
    - средний доход (от 50000 до 100000);
    - высокий доход (от 100000 до 350000);








In [ ]:
#функция категорирования по возрасту

def dob_years_cat(row):
    if row['dob_years'] <= 35:
        return 'до 35 лет'
    elif 36 <= row['dob_years'] < 45:
        return '36-45 лет'
    elif 46 <= row['dob_years'] <= 65:
        return '46-65 лет'
    else:
        return 'более 65 лет'
    
#функция категорирования по трудовому стажу

def years_employed_cat(row):
    if row['years_employed'] <= 10:
        return 'трудовой стаж до 10 лет'
    elif 10 < row['years_employed'] <= 30:
        return 'трудовой стаж 10-30 лет'
    else:
        return 'трудовой стаж более 30 лет'
    
#функция категорирования по количеству детей 

def children_cat(row):
    if row['children'] == 0:
        return 'нет детей'
    elif 1 <= row['children'] <= 2:
        return '1-2 ребенка'
    else:
        return 'многодетные'  
    
# функция категориварояни по доходу

def total_income_cat(row):
    if row['total_income'] <= 50000:
        return 'низкий доход'
    elif 50000 < row['total_income'] <= 100000:
        return 'средний доход'
    else:
        return 'высокий доход'
    
#этот столбец добавим для перевода значений возрата кредита в словесные значения

def debt_in_words(row):
    if row['debt'] == 0:
        return 'not_expired'
    else:
        return 'expired'    
    
    
#применим функции к таблице

borrowers ['dob_years_cat'] = borrowers.apply(dob_years_cat, axis=1)
borrowers['years_employed_cat'] = borrowers.apply(years_employed_cat, axis=1)
borrowers ['children_cat'] = borrowers.apply(children_cat, axis=1)
borrowers['total_income_cat'] = borrowers.apply(total_income_cat, axis=1)
borrowers['debt_in_words'] = borrowers.apply(debt_in_words, axis=1)


#выведем новую таблицу

borrowers.head(10)

,children,years_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category,dob_years_cat,years_employed_cat,children_cat,total_income_cat,debt_in_words
0,1,23,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,1,36-45 лет,трудовой стаж 10-30 лет,1-2 ребенка,высокий доход,not_expired
1,1,11,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,2,36-45 лет,трудовой стаж 10-30 лет,1-2 ребенка,высокий доход,not_expired
2,0,15,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,1,до 35 лет,трудовой стаж 10-30 лет,нет детей,высокий доход,not_expired
3,3,11,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,3,до 35 лет,трудовой стаж 10-30 лет,многодетные,высокий доход,not_expired
4,0,932,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,5,46-65 лет,трудовой стаж более 30 лет,нет детей,высокий доход,not_expired
5,0,2,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,1,до 35 лет,трудовой стаж до 10 лет,нет детей,высокий доход,not_expired
6,0,7,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,1,36-45 лет,трудовой стаж до 10 лет,нет детей,высокий доход,not_expired
7,0,0,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,3,46-65 лет,трудовой стаж до 10 лет,нет детей,высокий доход,not_expired
8,2,18,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,5,до 35 лет,трудовой стаж 10-30 лет,1-2 ребенка,средний доход,not_expired
9,0,5,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,1,36-45 лет,трудовой стаж до 10 лет,нет детей,высокий доход,not_expired


#### **Вывод**

Получили дата сет со следующими категориями новыми категориями:

- **purpose_category** - цель кредитования
- **years_employed_cat** - трудовой стаж в годах
- **children_cat** - количество детей
- **total_income_cat** - категория дохода
- **dob_years_cat**	- возрастная категория


Можем переходить к анализу и ответу на вопросы


## Ответы на поставленные вопросы

### Есть ли зависимость между наличием детей и возвратом кредита в срок?

Для выявления взаимосвязи наличия детей и возарата кредитных средств получим сводную таблицу и проверим гипотезу этой взаимосвязи:

In [ ]:
# получим сводную таблицу по столбцу - категории людей с детьми и невыполнению кредитных обязательств 


borr_pivot_child = borrowers.pivot_table(index=['children_cat'], columns=['debt_in_words'], values='debt', aggfunc='sum')

# назначим переменную для суммы всех просроченных кредитов

debt_expired = borrowers['debt'].sum()

# рассчитаем рейтинг невозврата в разрезе всех просроченных кредитов
borr_pivot_child ['ratio'] = borr_pivot_child ['expired']/debt_expired

# удалим вспомогательный столбец not_expired

borr_pivot_child = borr_pivot_child.drop('not_expired', axis=1)
borr_pivot_child.sort_values(by='expired')


debt_in_words,expired,ratio
children_cat,,
многодетные,39,0.022401
1-2 ребенка,638,0.366456
нет детей,1064,0.611143


#### **Вывод**

Можно сказать, что заёмщики с детьми дисциплинированы и чаще возвращают кредиты в срок. Таким образом, можно считать - что наличие детей - это преимущество и увеличивает вероятность у банка получать свои средства обратно в срок, что позволит потенциальным заемщикам увеличить свой бал. 


**Идеальный заемщик** с 1-2 детьми

### Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [ ]:
# Получим сводную таблицу по столбцу family_status


borr_pivot_fs = borrowers.pivot_table(index=['family_status'], columns=['debt_in_words'], values='debt', aggfunc='sum')

# рассчитаем рейтинг невозврата в разрезе всех просроченных кредитов

borr_pivot_fs ['ratio'] = borr_pivot_fs ['expired']/debt_expired

# удалим вспомогательный столбец not_expired


borr_pivot_fs = borr_pivot_fs.drop('not_expired', axis=1)
borr_pivot_fs.sort_values(by='expired', ascending=False)



debt_in_words,expired,ratio
family_status,,
женат / замужем,931,0.534750
гражданский брак,388,0.222860
не женат / не замужем,274,0.157381
в разводе,85,0.048823
вдовец / вдова,63,0.036186


#### **Вывод**

Можно сказать, что женатые/замужние чаще допускают просрочку, чем не женатые/не замужние, но категории можно приравнять, так как в случае с женатыми/замужними людьми - супруги выступают созаемщиками. На основании этого факта:

**Идеальный заемщик** состоит в браке

### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [ ]:
# Получим сводную таблицу по столбцу total_income_cat

borr_pivot_tot_income = borrowers.pivot_table(index=['total_income_cat'], columns=['debt_in_words'], values='debt', aggfunc='sum')

# рассчитаем рейтинг невозврата в разрезе всех просроченных кредитов

borr_pivot_tot_income ['ratio'] = borr_pivot_tot_income ['expired']/debt_expired

# удалим вспомогательный столбец not_expired

borr_pivot_tot_income = borr_pivot_tot_income.drop('not_expired', axis=1)
borr_pivot_tot_income.sort_values(by='expired', ascending=False)





debt_in_words,expired,ratio
total_income_cat,,
высокий доход,1387,0.796669
средний доход,331,0.190121
низкий доход,23,0.013211


#### **Вывод**

Получаем, что люди с высоким доходом чаще допускают просрочку кредита, нежеле люди со средним доходом. Люди с доходом как средний - более надежные заемщики.

**Идеальный заемщик** с доходом от 50 000,00 рублей до 100 000 рублей

###  Как разные цели кредита влияют на его возврат в срок?

In [ ]:
# Вызовем словарь
print(list_of_purpose)

{'недвиж': 1, 'авто': 2, 'образ': 3, 'опер': 4, 'свад': 5, 'строит': 6, 'ремо': 7, 'жил': 1}


In [ ]:
# Получим сводную таблицу по столбцу purpose_category

borr_pivot_purpose = borrowers.pivot_table(index=['purpose_category'], columns=['debt_in_words'], values='debt', aggfunc='sum')

# рассчитаем рейтинг невозврата в разрезе всех просроченных кредитов

borr_pivot_purpose ['ratio'] = borr_pivot_purpose ['expired']/debt_expired

#рассчитаем рейтинг для определения доли кредитов среди выданных
borr_pivot_purpose ['ratio_inside'] = (borr_pivot_purpose ['expired']/len(borrowers))*100

# удалим вспомогательный столбец not_expired

borr_pivot_purpose = borr_pivot_purpose.drop('not_expired', axis=1)
borr_pivot_purpose.sort_values(by='expired', ascending=False)




debt_in_words,expired,ratio,ratio_inside
purpose_category,,,
1,529,0.303848,2.465741
2,403,0.231476,1.878438
3,370,0.212522,1.724620
5,186,0.106835,0.866971
4,157,0.090178,0.731798
6,96,0.055141,0.447469


#### **Вывод**

Получаем, что чаще всего заемщики допускают просрочку кредитов, взятых с целью приобретения недвижимости, автомобиля и образование. Недвижимость и автомобиль, зачастую, берутся под залог - соответственно банк защищен в данных сделках, образование является более рисковым предметом кредитования.

**Идеальный заёмщик** берет в кредит недвижимость

###  Как возрастная категория влияют на возврат кредита в срок?

In [ ]:
# Получим сводную таблицу по столбцу dob_years_cat

borr_pivot_years_old = borrowers.pivot_table(index=['dob_years_cat'], columns=['debt_in_words'], values='debt', aggfunc='sum')

# рассчитаем рейтинг невозврата в разрезе всех просроченных кредитов

borr_pivot_years_old ['ratio'] = borr_pivot_years_old ['expired']/debt_expired

# удалим вспомогательный столбец not_expired


borr_pivot_years_old = borr_pivot_years_old.drop('not_expired', axis=1)
borr_pivot_years_old.sort_values(by='expired', ascending=False)




debt_in_words,expired,ratio
dob_years_cat,,
до 35 лет,706,0.405514
46-65 лет,533,0.306146
36-45 лет,427,0.245261
более 65 лет,75,0.043079


#### **Вывод**

Приходим к выводу, что чаще всего нарушают сроки выплат - молодые люди до 35 лет. Таким образом получаем:

**Идеальный заёмщик** в возрасте 36-45 лет

###  Как пол и семеное положение влияет на возврат кредита в срок?

In [ ]:
# Получим сводную таблицу по столбцу borr_pivot_gender

borr_pivot_gender = borrowers.pivot_table(index=['gender','family_status'], columns=['debt_in_words'], values='debt', aggfunc='sum')

# рассчитаем рейтинг невозврата в разрезе всех просроченных кредитов

borr_pivot_gender ['ratio'] = borr_pivot_gender ['expired']/debt_expired

# удалим вспомогательный столбец not_expired


borr_pivot_gender = borr_pivot_gender.drop('not_expired', axis=1)
borr_pivot_gender.dropna().sort_values(by='expired', ascending=False)



debt_in_words                 expired     ratio
gender family_status                           
F      женат / замужем          530.0  0.304423
M      женат / замужем          401.0  0.230327
F      гражданский брак         233.0  0.133831
M      не женат / не замужем    156.0  0.089604
       гражданский брак         155.0  0.089029
F      не женат / не замужем    118.0  0.067777
       в разводе                 61.0  0.035037
       вдовец / вдова            52.0  0.029868
M      в разводе                 24.0  0.013785
       вдовец / вдова            11.0  0.006318

#### Вывод

Чаще всего женщины допускают просрочку выплаты кредитных средств, что может быть связано с отпуском по беременности и родам. Сильного влияния пол не оказывает, но для банка гендер может стать важным при выдаче краткосрочных кредитов без залога имущества.

## Общий вывод

Провели исследование статистики о платёжеспособности клиентов. Пришли к следующим выводам:

1. Рисковые цели выдачей кредитных средств - покупка недвижимости и автомобиля. Однако, данные цели подкреплены залогом, стоимость высока, соответственно и выгода банка выше.
2. Люди, находящиеся в гражданском браке более ответственные, чем находящиеся в официальных отношениях, однако супруги, оформившие официальные отношения - выступают автоматически в качестве созаемщика, что защищает банк
3. Люди с высоким доходом являются менее дисциплинированными, чем заёмщики со средним доходом.
4. Заёмщики без детей - чаще допускают просрочку, нежеле люди с 1-2 ребенком в семье


**Попробуем нарисовать портет идеального заёмщика**:

Идеальный заёмщик в возрасте 36-45 лет, состоящий в официальном браке с 1-2 ребенком, доход средний 50 000 рублей - 100 000 рублей


Таким образом, семейное положение и количество детей **влияет** на факт погашения кредита

